In [ ]:
#Bert Embeddings
import pandas as pd
data = pd.read_csv('spam2.csv')
data.head()


,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
!pip install tensorflow

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
def get_bert_embeddings(text):
  inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
  with torch.no_grad():
    outputs = model(**inputs)
    cls_embeddings = outputs.last_hidden_state[:, 0, :]
    return  cls_embeddings

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
import torch

In [ ]:
texts = data['text'].tolist()
labels = data['label'].map({'ham': 0, 'spam': 1}).tolist()

In [ ]:
embeddings = []
for text in texts:
  embedding = get_bert_embeddings(text)
  embeddings.append(embedding)

In [ ]:
embeddings

[tensor([[-1.5131e-01, -3.2292e-01,  1.8985e-01, -1.8147e-01, -3.3117e-01,
          -2.8334e-01,  5.2693e-01,  3.8858e-01,  2.2110e-01, -3.4960e-01,
          -3.1281e-01, -4.5458e-01, -3.3461e-01,  4.8413e-01,  4.2673e-01,
           5.4971e-01, -1.4162e-01,  2.8117e-01,  6.8661e-02, -2.2515e-01,
          -2.8676e-01, -3.8071e-01,  6.8041e-02,  2.5098e-01,  1.6956e-01,
          -1.3872e-01,  4.3515e-02, -1.7103e-01, -1.6701e-01,  3.3736e-01,
           6.6232e-02,  6.0870e-01, -6.4672e-01,  1.5898e-01,  3.2143e-01,
           2.4391e-01,  5.7457e-01,  7.3135e-02,  8.5454e-02,  2.8577e-01,
          -2.7645e-01,  4.2477e-01,  3.6938e-01, -1.6104e-01, -9.4477e-03,
          -5.4210e-01, -3.4944e+00, -9.4472e-03, -5.7548e-01, -6.9572e-01,
           7.9051e-02, -2.1258e-01,  1.5563e-01,  4.7859e-01,  1.8580e-01,
           4.8720e-01, -1.6174e-01,  1.1350e-02,  2.4365e-01, -6.3855e-02,
           8.1738e-01,  1.0882e-01,  3.7179e-01,  3.8512e-01,  1.4748e-02,
           1.9756e-01,  3

In [ ]:
X = np.array(embeddings)
y = np.array(labels)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


In [ ]:
X_train = X_train.reshape(X_train.shape[0],-1)
X_test = X_test.reshape(X_test.shape[0],-1)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train,y_train)

LogisticRegression()

In [ ]:
clf.score(X_train,y_train)

1.0

In [ ]:
text = " free lottery weekly camp"
bert_embeddings = get_bert_embeddings(text)
prediction = clf.predict(bert_embeddings)
print(f"Prediction :{prediction}")

Prediction :[1]


In [ ]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
from sentence_transformers import SentenceTransformer,util

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
file_path = '/content/sample1.txt'

In [ ]:
with open(file_path, 'r') as file:
    text_data = file.readlines()

In [ ]:
text_data = [line.strip() for line in text_data if line.strip()]

In [ ]:
text_embeddings = model.encode(text_data, convert_to_tensor=True)
text_embeddings

tensor([[-6.3760e-02, -7.0007e-02,  4.0222e-02,  ...,  1.3160e-01,
          6.1766e-02, -4.3437e-02],
        [-3.6404e-02, -4.4544e-02, -1.0446e-01,  ...,  1.3567e-01,
         -1.3651e-03,  5.9077e-02],
        [-1.1114e-01, -1.2784e-01,  1.8348e-04,  ...,  1.0968e-01,
          2.5579e-02,  2.0339e-03],
        [-1.9196e-01, -4.3118e-02,  3.6412e-02,  ...,  1.1358e-01,
          5.5202e-02,  7.6192e-02]])

In [ ]:
question = " what is argmax?"

In [ ]:
question_embeddings = model.encode(question, convert_to_tensor=True)

In [ ]:
question_embeddings

tensor([-5.8451e-02, -4.7675e-02, -1.2315e-01,  1.6038e-03, -1.3751e-02,
        -4.5500e-02,  9.8552e-02,  3.5621e-02, -9.2739e-02,  5.4401e-02,
         2.5710e-02,  1.0192e-01,  1.2448e-02,  9.2157e-03, -1.1260e-01,
        -1.2386e-02,  1.7147e-02, -2.8883e-02,  3.6798e-02, -5.5561e-02,
        -1.8413e-02,  2.1266e-02, -4.2417e-02, -4.1653e-02,  2.4054e-03,
         3.9770e-02, -3.5603e-02,  5.5787e-02,  5.0510e-02, -4.4374e-02,
         2.4911e-02,  1.7832e-02,  1.3619e-01, -8.2147e-03, -2.6155e-02,
         1.0665e-01, -7.6310e-02, -6.1607e-02, -4.5566e-02, -2.4532e-02,
        -5.9952e-02,  8.9394e-03, -1.9743e-02, -5.9903e-02,  2.2560e-02,
         2.6847e-02, -1.9231e-02,  6.4573e-03, -6.0519e-02,  1.2170e-01,
        -1.2126e-03,  7.5375e-03, -1.4702e-02,  1.8900e-02, -3.0599e-03,
        -2.6591e-02, -5.6094e-02, -9.1850e-02, -5.5362e-03, -2.6911e-02,
         3.1918e-02, -2.7801e-02, -7.6515e-02,  3.6437e-02, -7.2244e-03,
        -3.9678e-02,  5.3575e-02, -3.6199e-02,  8.0

In [ ]:
similarity_scores = util.pytorch_cos_sim(question_embeddings, text_embeddings)

In [ ]:
best_match_idx = similarity_scores.argmax()

In [ ]:
answer = text_data[best_match_idx]
print(answer)

2)In mathematics, argmax is the argument of a function that produces the maximum output value.


In [ ]:
def q_a(question):
  question_embeddings = model.encode(question, convert_to_tensor=True)
  similarity_scores = util.pytorch_cos_sim(question_embeddings, text_embeddings)
  best_match_idx = similarity_scores.argmax()
  answer = text_data[best_match_idx]
  return answer

In [ ]:
q_a('what is a rnn')

'3)RNN stands for recurrent neural network, which is a type of artificial neural network that processes sequential data.'

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 80.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing in

In [ ]:
import gradio as gr

In [ ]:
gr.Interface(fn=q_a,inputs='text',outputs='text').launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5cad84bcfd5e95b990.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
